*Load packages and data:*

In [1]:
%run stdPackages.ipynb
read = {'variables': ['Fundamentals', 'Load', 'Generators_Other'], 
        'variable2D': ['Generators_FuelMix'],
        'maps': ['Generators_Categories']}
db = dbFromWB(os.path.join(d['data'],'E1.xlsx'), read)
readSets(db)

# Exercise 1: Getting to know the ```mBasic``` model class

The objective of the first exercise is to get to know the models used throughout the course and learn how to interact with the code. Thus, the intuitive gains from the exercise might be limited. The exercise builds on the ```mBasic``` model class; you can find helpful information on this in the following:
* The note "*Models for Energy Economics*" describes the model in section 2.
* The notebook [M_mBasic](M_mBasic.ipynb) includes some description of the model class and shows how to interact with the model class and run simple experiments.
* The notebook [E1_SolutionGuide](E1_SolutionGuide.ipynb) provides a short solution to the problems posted in the current notebook.

*Note: If you are not familiar with Python and specifically the package ```pandas```, you may benefit from spending some time dissecting the code used in the solution guide instead of trying to solve the exercises on your own.*

## The setup:

We consider a ```mBasic``` model where 5 different electricity producing plants contribute to satisfy a fixed load of $100$ GJ. 4 of the plants use a mix of four fuels: Coal, natural gas, biomass, or waste. The last plant produces wind power. The parameter ```'FuelMix'``` that is stored in the ```db``` database shows the input shares: 

In [2]:
db['FuelMix'].unstack('BFt') # note that you access the symbols in the database 'db' using the slicing syntax '[]'

BFt,Coal,NatGas,BioMass,Waste
id,,,,
id1,0.9,0.1,0,0
id2,0.1,0.9,0,0
id3,0,0,1,0
id4,0,0,0,1.15
id5,0,0,0,0


### Problem 1: Compute the marginal costs of operation for the five plants.

The plants pays a price (```FuelPrice```) for the fuel inputs and pays a tax (```EmissionTax```)  on the CO2 it emits. Furthermore, the plant has 'other costs' of operation (```OtherMC```). The marginal cost of operation for the five plants are then computed as:
$$\begin{align}
 mc_{id} = \text{OtherMC}_{id}+\sum_{BFt} \mu_{BFt}^{id}\left[\text{FuelPrice}_{BFt}+\text{EmissionTax}\cdot \text{EmissionIntensity}_{BFt}\right],
\end{align}$$

*Compute the marginal costs for the five plants. Check that your result is equivalent to the result from the predefined method ```lpModels.mc(db)```.*

### Problem 2: Identify the marginal producer and the system marginal costs

The *marginal producer* and *system marginal costs* are defined in the note "*Models for Energy Economics*". To identify the two concepts, we first need to solve the model. Doing this is very straightforward using the ```mBasic``` model class. Specifically, we use two simple steps: (1) Initialize the model with the database, and (2) solve:

In [3]:
m = mBasic.mBasic(db) # initialize using the database 'db' from above.
m.solve() # solve the model; should return a short printed statement that optimization is succesful.

Solution status 0: Optimization terminated successfully.


*Identify the marginal producer (a specific id) and the system marginal costs of this model.*

*Hint: The optimal production is referred to as ```'Generation'```, and the generation capacity is called ```GeneratingCapacity```.*

### Problem 3: A shock to tax on CO2 emissions

*Increase the tax on CO2 emissions to respectively $50$ and $100$ EUR/Tonnes CO2. Report the effect on emissions and explain briefly.*

*Hint 1: When re-solving the model the results in the 'db' are overwritten. To save the first results, create some type of copy before resolving, for instance using the copy method:*

In [4]:
db_BeforeShock = m.db.copy()

*Hint 2: To recompute the marginal costs before solving the model again, you can pass the following option to the solve statment*

In [5]:
m.solve(preSolve = {'recomputeMC': True})

Solution status 0: Optimization terminated successfully.


*Hint 3: Emissions are automaticaly stored as the symbol 'Emissions' when the model is solved. So, you don't have to compute them manually, but just access them:*

In [6]:
m.db['Emissions']

EmissionType
CO2    5.669729
dtype: object

*Revert to the original database before carrying out the other shocks:*

In [7]:
m.db = db_BeforeShock

### Problem 4: Abating emissions using wind power

In the final task, we will practice using the predefined method ```loopSolveExtract``` to run simple experiments. This method is defined to create the following simple type of experiments in a loop: For each step in the loop:
* update some parameter value,
* re-solve the model,
* extract some specific parts of the solution that you are interested in.

*Solve the model on a grid of different levels of installed wind power: From 10 to 50 with 41 gridpoints. Extract the solution for emissions and fuel consumption on this same grid. Plot the solutions and comment briefly on how effective a mandate to install additional wind energy is at abating emissions in the current model.*

*Hint 1: The notebook [M_mBasic](M_mBasic.ipynb) has an example of the syntax needed to use the ```loopSolveExtract```. Use this as inspiration for how to apply the method in this case.*

*Hint 2: The auxiliary function ```addGrid``` can create a grid for us based on the first and the final value on the grid. The following syntax can be used to create a grid 'windGrid' from 10-50 GJ with 41 gridpoints:*

In [8]:
loop = pd.Index(range(41), name = 'loop')
v0 = pd.Series(10, index = pd.Index(['id5'], name = 'id'))
vT = pd.Series(50, index = pd.Index(['id5'], name = 'id'))
windGrid = addGrid(v0,vT,loop,'GeneratingCapacity') # inputs are (1) first value in grid, (2) final value, (3) loop, (4) name of symbol